This tutorial can be downloaded as part of the [Wallaroo Tutorials repository]([wallaroo-model-deploy-and-serve/keras-upload-tutorials](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-model-deploy-and-serve/keras-upload-tutorials)).

## Wallaroo Model Upload via the Wallaroo SDK: TensorFlow keras Sequential Single IO

The following tutorial demonstrates how to upload a TensorFlow keras Sequential Single IO model to a Wallaroo instance.

### Tutorial Goals

Demonstrate the following:

* Upload a TensorFlow keras Sequential Single IO to a Wallaroo instance.
* Create a pipeline and add the model as a pipeline step.
* Perform a sample inference.

### Prerequisites

* Wallaroo Version 2023.2.1 or above instance.

### References

* [Wallaroo MLOps API Essentials Guide: Model Upload and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-api-essential-guide/wallaroo-mlops-api-essentials-guide-model-uploads/)
* [Wallaroo API Connection Guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-connection-guide/)
* [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [8]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline   import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework
from wallaroo.object import EntityNotFoundError

import pyarrow as pa
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

import datetime

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [9]:
wl = wallaroo.Client()

### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.

In [10]:
workspace_name = f'keras-sequential-single-io'
pipeline_name = f'keras-sequential-single-io'

model_name = 'keras-sequential-single-io'
model_file_name = 'models/model-auto-conversion_keras_single_io_keras_sequential_model.h5'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [11]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Configure Data Schemas

The following parameters are required for TensorFlow keras models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a TensorFlow Keras model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Upload Method Optional, TensorFlow keras model Required*) | Set as the `Framework.KERAS`. |
|`input_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, TensorFlow Keras model Required*) | The input schema in Apache Arrow schema format. |
|`output_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, TensorFlow Keras model Required*) | The output schema in Apache Arrow schema format. |
| `convert_wait` | `bool` (*Upload Method Optional, TensorFlow model Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

In [12]:
input_schema = pa.schema([
    pa.field('input', pa.list_(pa.float64(), list_size=10))
])
output_schema = pa.schema([
    pa.field('output', pa.list_(pa.float64(), list_size=32))
])

### Upload Model

The model will be uploaded with the framework set as `Framework.KERAS`.

In [13]:
framework=Framework.KERAS

model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=framework, 
                        input_schema=input_schema, 
                        output_schema=output_schema)
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime.
Model is attempting loading to a native runtime..incompatible

Model is pending loading to a container runtime.
Model is attempting loading to a container runtime.................successful

Ready


Name,keras-sequential-single-io
Version,9ab17dc8-50e0-43b4-b0b9-76084f593d0c
File Name,model-auto-conversion_keras_single_io_keras_sequential_model.h5
SHA,f7e49538e38bebe066ce8df97bac8be239ae8c7d2733e500c8cd633706ae95a8
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2024.1.0-main-4898
Architecture,x86
Acceleration,none
Updated At,2024-11-Apr 21:11:26


In [14]:
model.config().runtime()

'flight'

### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

In [15]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [16]:
# clear the pipeline if used in a previous tutorial
pipeline.undeploy()
pipeline.clear()
pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.0.179',
   'name': 'engine-6498c8b7df-8p92g',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'keras-sequential-single-io',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'keras-sequential-single-io',
      'sha': 'f7e49538e38bebe066ce8df97bac8be239ae8c7d2733e500c8cd633706ae95a8',
      'status': 'Running',
      'version': '9ab17dc8-50e0-43b4-b0b9-76084f593d0c'}]}}],
 'engine_lbs': [{'ip': '10.28.3.156',
   'name': 'engine-lb-d7cc8fc9c-2vjds',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': [{'ip': '10.28.2.39',
   'name': 'engine-sidekick-keras-sequential-single-io-130-6bb87f86cf-rppkx',
   'status': 'Running',
   'reason': None,
   'details': [],
   'statuses': '\n'}]}

### Run Inference

A sample inference will be run.  First the pandas DataFrame used for the inference is created, then the inference run through the pipeline's `infer` method.

In [17]:
input_data = np.random.rand(10, 10)
mock_dataframe = pd.DataFrame({
    "input": input_data.tolist()
})
mock_dataframe

,input
0,"[0.41183803040941347, 0.3632065124375583, 0.69..."
1,"[0.793574167387814, 0.08092450622057146, 0.419..."
2,"[0.3514529261775484, 0.6279985186708846, 0.550..."
3,"[0.40041424660351443, 0.5146023074698263, 0.14..."
4,"[0.7934727525267351, 0.054854918994684865, 0.7..."
5,"[0.36116101274919077, 0.7400533825241119, 0.23..."
6,"[0.5622486141257285, 0.5558365849386399, 0.669..."
7,"[0.7973780566188386, 0.08904024430789714, 0.73..."
8,"[0.30320091924850945, 0.7632289957226438, 0.25..."
9,"[0.007144374324637104, 0.8477130982856738, 0.7..."


In [18]:
pipeline.infer(mock_dataframe)

,time,in.input,out.output,anomaly.count
0,2024-04-11 21:12:01.296,"[0.4118380304, 0.3632065124, 0.6955939757, 0.9...","[0.02687848, 0.03252433, 0.036281176, 0.030184...",0
1,2024-04-11 21:12:01.296,"[0.7935741674, 0.0809245062, 0.419346271, 0.36...","[0.015604752, 0.036830872, 0.04238343, 0.03049...",0
2,2024-04-11 21:12:01.296,"[0.3514529262, 0.6279985187, 0.5509772767, 0.4...","[0.024944814, 0.0231208, 0.040473048, 0.021774...",0
3,2024-04-11 21:12:01.296,"[0.4004142466, 0.5146023075, 0.1445976063, 0.7...","[0.023686556, 0.031468812, 0.030059567, 0.0236...",0
4,2024-04-11 21:12:01.296,"[0.7934727525, 0.054854919, 0.7966853828, 0.87...","[0.018660381, 0.036466595, 0.046900153, 0.0256...",0
5,2024-04-11 21:12:01.296,"[0.3611610127, 0.7400533825, 0.2308874776, 0.2...","[0.024499405, 0.02814265, 0.033328447, 0.03579...",0
6,2024-04-11 21:12:01.296,"[0.5622486141, 0.5558365849, 0.6697683779, 0.1...","[0.03647522, 0.019568041, 0.045952123, 0.02628...",0
7,2024-04-11 21:12:01.296,"[0.7973780566, 0.0890402443, 0.7324945362, 0.3...","[0.027356947, 0.026283065, 0.039344788, 0.0287...",0
8,2024-04-11 21:12:01.296,"[0.3032009192, 0.7632289957, 0.257553301, 0.05...","[0.03398794, 0.021575732, 0.048853625, 0.02785...",0
9,2024-04-11 21:12:01.296,"[0.0071443743, 0.8477130983, 0.7131040011, 0.4...","[0.03118782, 0.025181519, 0.039937932, 0.03172...",0


### Undeploy Pipelines

With the tutorial complete, the pipeline is undeployed to return the resources back to the cluster.

In [19]:
pipeline.undeploy()

name,keras-sequential-single-io
created,2024-04-11 19:20:57.049755+00:00
last_updated,2024-04-11 21:11:29.616951+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"f27d2f0c-3dc1-428a-82b8-9f29a476a198, da2e3dbd-b5c7-4e89-b5b8-eec811801a69, 949e7221-5719-4471-b620-4a8a18ec5a9a, b49aa9cd-4c84-49c7-a3de-27cc561d7b2c, d9366db7-78ca-4c4a-be99-2873825ef104"
steps,keras-sequential-single-io
published,False
